# Global averaging

In [183]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.signal import detrend
from matplotlib import pyplot as plt
from scipy import signal
import pandas as pd
import xarray as xr
import intake
import pprint
import util 
import os

col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)

file = 'available_tas_data_cloud_june12th_2020.txt'

In [2]:
# load table:
data_table = pd.read_table(file,index_col=0)
models_used = data_table['piControl (yrs)'].index
print(models_used)

Index(['IPSL-CM6A-LR', 'MRI-ESM2-0', 'MPI-ESM1-2-LR', 'TaiESM1',
       'AWI-CM-1-1-MR', 'AWI-ESM-1-1-LR', 'BCC-CSM2-MR', 'BCC-ESM1',
       'CAMS-CSM1-0', 'FGOALS-f3-L', 'FGOALS-g3', 'IITM-ESM', 'CanESM5',
       'CanESM5-CanOE', 'CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1',
       'ACCESS-ESM1-5', 'ACCESS-CM2', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA',
       'EC-Earth3', 'EC-Earth3-LR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR',
       'FIO-ESM-2-0', 'MPI-ESM-1-2-HAM', 'INM-CM4-8', 'INM-CM5-0',
       'MIROC-ES2L', 'MIROC6', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM',
       'UKESM1-0-LL', 'MPI-ESM1-2-HR', 'GISS-E2-1-G', 'GISS-E2-1-G-CC',
       'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM',
       'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM',
       'KACE-1-0-G', 'GFDL-CM4', 'GFDL-ESM4', 'NESM3', 'SAM0-UNICON', 'CIESM',
       'MCM-UA-1-0'],
      dtype='object')


In [33]:
# load functions:
def area_weights(lat_bnds, lon_bnds): 
    # computes exact area weigths assuming earth is a perfect sphere
    lowerlats = np.radians(lat_bnds[:,0]); upperlats = np.radians(lat_bnds[:,1])
    difflon = np.radians(np.diff(lon_bnds[0,:])) # if the differences in longitudes are all the same
    areaweights = difflon*(np.sin(upperlats) - np.sin(lowerlats));
    areaweights /= areaweights.mean()
    return areaweights # list of weights, of same dimension as latitude

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def leap_year(year, calendar='standard'):
    """Determine if year is a leap year"""
    leap = False
    if ((calendar in ['standard', 'gregorian',
        'proleptic_gregorian', 'julian']) and
        (year % 4 == 0)):
        leap = True
        if ((calendar == 'proleptic_gregorian') and
            (year % 100 == 0) and
            (year % 400 != 0)):
            leap = False
        elif ((calendar in ['standard', 'gregorian']) and
                 (year % 100 == 0) and (year % 400 != 0) and
                 (year < 1583)):
            leap = False
    return leap

# function copied from: http://xarray.pydata.org/en/stable/examples/monthly-means.html
def get_dpm(time, calendar='standard'):
    """
    return a array of days per month corresponding to the months provided in `months`
    """
    month_length = np.zeros(len(time), dtype=np.int)

    cal_days = dpm[calendar]

    for i, (month, year) in enumerate(zip(time.month, time.year)):
        month_length[i] = cal_days[month]
        if leap_year(year, calendar=calendar) and month == 2:
            month_length[i] += 1
    return month_length

# days per month:
dpm = {'noleap': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       'julian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31], ##### I think this should be correct
       'proleptic_gregorian': [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31],
       '360_day': [0, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]
      }

def compute_day_weights(ds, calendar = 'noleap', first_month = 1): # new function
    month_length = xr.DataArray(get_dpm((ds.time.to_index()), calendar=ds_calendar), coords=[ds.time], name='month_length')
    
    ##### This code is only tested for noleap so far #####
    if first_month == 1:
        norm_by_annual = month_length.groupby('time.year').mean('time') # make annual mean
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual.values[i], 12) for i in range(len(norm_by_annual.values))])
    else: 
        norm_by_annual = np.array([month_length[i*12:(i+1)*12].mean() for i in range(int(len(month_length)/12))])
        norm_by_monthly = np.concatenate([np.tile(norm_by_annual[i], 12) for i in range(len(norm_by_annual))])
        
    weights = month_length/norm_by_monthly
    # normalized to have mean 1
    return weights 

def calendar_check(model):
    # Time formats for piControl, found from manual check:
    if model in ['TaiESM1', 'BCC-CSM2-MR', 'BCC-ESM1', 'CAMS-CSM1-0', 'CAS-ESM2-0', 'FGOALS-f3-L', 'FGOALS-g3', 'CanESM5', 'CanESM5-CanOE', 'E3SM-1-0', 'E3SM-1-1', 'E3SM-1-1-ECA', 'FIO-ESM-2-0', 'INM-CM4-8', 'INM-CM5-0', 'GISS-E2-1-G', 'GISS-E2-1-G-CC', 'GISS-E2-1-H', 'GISS-E2-2-G', 'CESM2', 'CESM2-FV2', 'CESM2-WACCM', 'CESM2-WACCM-FV2', 'NorCPM1', 'NorESM1-F', 'NorESM2-LM', 'NorESM2-MM', 'GFDL-CM4', 'SAM0-UNICON', 'GFDL-ESM4', 'CIESM', 'MCM-UA-1-0']:
        ds_calendar = 'noleap'
    elif model in ['EC-Earth3', 'CNRM-CM6-1', 'IPSL-CM6A-LR', 'MIROC-ES2L', 'MIROC6', 'NESM3']: # 'IPSL-CM6A-LR':'piClim-4xCO2','piClim-control' says noleap calendar
        ds_calendar = 'gregorian'
    elif model in ['AWI-CM-1-1-MR', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'ACCESS-ESM1-5', 'ACCESS-CM2', 'MPI-ESM-1-2-HAM', 'MPI-ESM1-2-LR', 'MPI-ESM1-2-HR', 'EC-Earth3-LR']:
        ds_calendar = 'proleptic_gregorian'
    elif model in ['UKESM1-0-LL', 'HadGEM3-GC31-LL', 'HadGEM3-GC31-MM', 'CNRM-ESM2-1', 'KACE-1-0-G', 'MRI-ESM2-0']:
        ds_calendar = '360_day'
        if model in ['CNRM-ESM2-1', 'MRI-ESM2-0']:
            print('piControl is 360_day, the other experiments unknown')
    elif model in ['IITM-ESM']:
        ds_calendar = 'julian'
    elif model in ['AWI-ESM-1-1-LR', 'CNRM-CM6-1-HR', 'EC-Earth3', 'EC-Earth3-LR']:
        #ds_calendar = 'datetime64'
        print('not 100% sure what calendar this model has, but a guess is made based on other models from same institution')
        if model in ['AWI-ESM-1-1-LR']:
            print('calendar is likely proleptic gregorian')
            ds_calendar = 'proleptic_gregorian'
        elif model in ['CNRM-CM6-1-HR']:
            print('calendar is likely gregorian')
            ds_calendar = 'gregorian'
    return ds_calendar

# calendar double checked for models:
# ACCESS-CM2: proleptic_gregorian

# 'IPSL-CM6A-LR', 'abrupt-4xCO2': gregorian
# 'EC-Earth3-LR', 'piControl': proleptic_gregorian

    

# Choose model, exp, member

In [133]:
#model = models_used[10];
#model = 'EC-Earth3-LR'
#model = 'IPSL-CM6A-LR'
model = 'ACCESS-CM2'

print(model)
ds_calendar = calendar_check(model)
print(ds_calendar, 'calendar')

#check what experiments are available for var
var = 'tas'
#cat = col.search(experiment_id = ['piControl', 'abrupt-4xCO2', 'abrupt-2xCO2', 'abrupt-0p5xCO2', 'historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585'], source_id = model, variable_id=var, table_id='Amon') 
cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 
#cat = col.search(experiment_id = 'abrupt-4xCO2', source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon') 


#cat.df
pd.set_option('display.max_colwidth', -1)
cat.df['zstore']

ACCESS-CM2
proleptic_gregorian calendar


26856    gs://cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/Amon/rlut/gn/
26860    gs://cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/Amon/rsdt/gn/
26863    gs://cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/Amon/rsut/gn/
26867    gs://cmip6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/piControl/r1i1p1f1/Amon/tas/gn/ 
Name: zstore, dtype: object

In [176]:
#exp = 'piControl'
#exp = 'abrupt-4xCO2'
#exp = 'abrupt-2xCO2'
#exp = 'abrupt-0p5xCO2'
#exp = 'historical'
#exp = 'ssp126'
#exp = 'ssp245'
#exp = 'ssp370'
#exp = 'ssp585'
#exp = 'piClim-4xCO2'
exp = 'piClim-control'
#exp = 'piClim-histall'


if model == 'UKESM1-0-LL':
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'MOHC')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'NIMS-KMA')
if model == 'MPI-ESM1-2-HR': # select institution if there are two groups
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DKRZ') 
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', institution_id = 'DWD') 
elif model in ['MPI-ESM1-2-LR', 'IPSL-CM6A-LR', 'MRI-ESM2-0', 'EC-Earth3'] and exp in ['piControl']: # problem when loading more than one member simultaneously, so specify member_id:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon', member_id = 'r2i1p1f1')
else:
    cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
    #cat = col.search(experiment_id = exp, source_id = model, variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon',
    #                 member_id = ['r2i1p1f1','r3i1p1f1','r4i1p1f1','r5i1p1f1','r6i1p1f1','r7i1p1f1','r8i1p1f1','r9i1p1f1','r10i1p1f1','r11i1p1f1','r12i1p1f1']) 
    
    
if 'dset_dict' in locals():
    del dset_dict
if 'ds_exp' in locals():
    del ds_exp
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})
#for key in ['CMIP.MOHC.UKESM1-0-LL.historical.Amon.gn']:
for key in dset_dict.keys():
    ds_exp = dset_dict[key]
    if 'members_sorted' in locals():
        del members_sorted
    members_sorted = ds_exp.member_id.sortby(ds_exp.member_id)
    
if model == 'MCM-UA-1-0':
    ds_exp = ds_exp.rename({'longitude': 'lon','latitude': 'lat'}) 
        
for member in members_sorted:
    print(member.values)
    
# write out data variables, to check that we have all we want
ds_exp.data_vars

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
r1i1p1f1


Data variables:
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(192, 2), meta=np.ndarray>
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(144, 2), meta=np.ndarray>
    time_bnds  (time, bnds) object dask.array<chunksize=(360, 2), meta=np.ndarray>
    rlut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 180, 144, 192), meta=np.ndarray>
    rsdt       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 180, 144, 192), meta=np.ndarray>
    rsut       (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 180, 144, 192), meta=np.ndarray>
    height     float64 ...
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 180, 144, 192), meta=np.ndarray>

In [177]:
#ds_exp['time']

In [178]:
# latitude boundaries are not just simple means of latitude coordinates..
# reconstruct lon_bnds for French models missing this coordinate
#lon_bnds_1dim = (ds_exp.lon[1:].values + ds_exp.lon[0:-1].values)/2
#lon_bnds_2dim = np.array([[lon_bnds_1dim[i], lon_bnds_1dim[i+1]] for i in range(len(ds_exp.lon)-2)]) # middle values
#lon_bnds_2dim = np.append([[-lon_bnds_2dim[0,0], lon_bnds_2dim[0,0]]],lon_bnds_2dim, axis = 0) # add first values
#lon_bnds_2dim = np.append(lon_bnds_2dim, [[lon_bnds_2dim[-1,1], (360+ds_exp.lon[0]+ds_exp.lon[-1])/2]], axis = 0) # add last values

#ds_exp['tas'][0,0,0,0:10].values
#ds_exp['rlut'][0,0,0,0:10].values

#for member in members_sorted:
#    print(member.values)
#
#    ds = ds_exp.sel(member_id = member)
#    print(ds_exp.sizes)
#    print(ds.lon_bnds[0:10].values)
#    print(ds.lat_bnds[0:10].values)

#ds_exp.grid

#ds_exp.rlut.isel(time = 10).plot()
#dset_dict.keys()

# testing model EC earth3
#ds_test = ds_exp.sel(member_id = 'r4i1p1f1')
#ds_test['tas'].isel(time=0).values[0]
#import sys
#np.set_printoptions(threshold=sys.maxsize)
#ds_test['tas'].isel(time=0).values[0].shape
#ds_test.lon

#differences = norm_areas-area_w
#differences
#plt.plot(differences[1:-1])
#col.search(source_id = model, variable_id=['areacella']).df

In [179]:
ds_exp.time.encoding['calendar']

'proleptic_gregorian'

In [180]:
if model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
    # if computing area-weights from areacella instead:
    #area_cat= col.search(source_id = model, variable_id=['areacella'])
    area_cat = col.search(experiment_id = 'piControl', source_id = model, variable_id=['areacella'])
    area_dset_dict = area_cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

    for key in area_dset_dict.keys():
        area_ds = area_dset_dict[key]
    areas = area_ds['areacella'].values[0,:,0]
    norm_areas = areas/areas.mean()

# loop through members
#for member in [members_sorted.sel(member_id = 'r1i1p1f1')]:
for member in members_sorted:
    print(member.values)

    ds = ds_exp.sel(member_id = member)
    
    ds_calendar = ds.time.encoding['calendar']
    unit = ds.time.encoding['units']
    firstmonth = ds_exp.time.to_index().month[0]
    
    print(ds_calendar, 'calendar')
    print(unit)
    print('first month of dataset is:', firstmonth)
    
    # compute weights for average
    if model == 'NorCPM1' and exp == 'historical':
        area_w = area_weights(ds.lat_bnds.values[0,:,:], ds.lon_bnds.values)
    elif model in ['CNRM-CM6-1', 'CNRM-CM6-1-HR', 'CNRM-ESM2-1', 'IPSL-CM6A-LR']: # French models missing lat_bnds and lon_bnds coordinates
        area_w = norm_areas
        #print('maximum difference in area-weights is:', np.max(np.abs(norm_areas - area_w)))
        
        # differences in global means are noticeable from the third decimal for the low-res model BCC-ESM1
        # for this model maximum difference in area-weights is: 0.003299741144367374
        
        # differences in global means are noticeable from the 5th decimal for the model CESM2
        # for this model maximum difference in area-weights is: 1.2547306571519812e-07
    else:
        area_w = area_weights(ds.lat_bnds.values, ds.lon_bnds.values)
        
    day_weights = compute_day_weights(ds, calendar = ds_calendar, first_month = firstmonth)

    varlist = ['tas', 'rlut', 'rsut', 'rsdt']
    
    for variable in varlist:
        print(variable)
        data = ds[variable]
                
        # global average
        area_avg = (data.transpose('time', 'lon', 'lat') * area_w).mean(dim=['lon', 'lat'])

        # annual average
        day_weighted_avg = area_avg*day_weights
        if firstmonth == 1:
            annualmean = day_weighted_avg.groupby('time.year').mean('time')
        else:
            annualmean_array = np.array([day_weighted_avg[i*12:(i+1)*12].mean() for i in range(int(len(day_weighted_avg)/12))])
            annualmean = xr.DataArray(annualmean_array)
            
        if variable == varlist[0]:
            # create dataframe for storing all results
            df = pd.DataFrame(annualmean.values, columns = [variable])
        else:
            df_col = pd.DataFrame(annualmean.values, columns = [variable])
            df = pd.merge(df, df_col, left_index=True, right_index=True, how='outer')
            
    filename = model + '_' + exp + '_' + str(member.values) + '_means.txt'
    file = os.path.join('../Processed_data/Global_annual_means/', model, exp, filename)
    #if member == members_sorted[0]: # create directory for first member
    #    os.makedirs(os.path.dirname(file), exist_ok=False)

    df.to_csv(file)
    

r1i1p1f1
proleptic_gregorian calendar
hours since 0960-01-16 12:00:00.000000
first month of dataset is: 1
tas
rlut
rsut
rsdt


In [182]:
#area_ds
df

,tas,rlut,rsut,rsdt
0,287.092977,242.285769,97.995931,340.349984
1,287.025986,242.214299,98.038223,340.318251
2,287.021995,242.189091,97.778931,340.318251
3,287.026616,242.308661,98.087934,340.318251
4,287.036215,242.133072,97.819375,340.349984
5,287.020650,242.214669,98.069151,340.318251
6,287.036991,242.132553,97.829216,340.318251
7,287.015563,241.987156,97.805878,340.318251
8,287.014245,241.981164,98.244990,340.349984
9,287.028331,242.159191,97.953704,340.318251


In [67]:
#area_w_df = df
#area_w_df

,tas,rlut,rsut,rsdt
0,286.978921,240.510313,98.533597,340.2065
1,287.025035,240.350015,98.631853,340.2065
2,286.989021,240.065237,98.142325,340.2065
3,287.022557,240.146973,98.663816,340.2065
4,287.044452,240.434060,98.632153,340.2065
5,286.996617,240.034233,98.715008,340.2065
6,287.003279,240.427962,98.816115,340.2065
7,287.034542,240.269484,98.952014,340.2065
8,286.979799,239.991135,98.544066,340.2065
9,286.936304,240.370579,98.620144,340.2065


In [70]:
#area_w_df - df

,tas,rlut,rsut,rsdt
0,9.802239e-06,5.428059e-06,8.223801e-07,0.000003
1,-1.673645e-06,1.092329e-05,3.766649e-06,0.000003
2,2.416901e-06,5.882505e-06,3.584533e-06,0.000003
3,-5.392528e-07,5.601185e-06,3.180968e-06,0.000003
4,3.369298e-06,-2.622784e-06,3.102178e-07,0.000003
5,7.886061e-06,1.028636e-05,3.713612e-06,0.000003
6,-4.010063e-07,2.861549e-06,-4.321406e-07,0.000003
7,6.694797e-06,1.839986e-06,2.446704e-06,0.000003
8,6.289548e-06,9.916292e-06,1.129158e-06,0.000003
9,5.357593e-06,5.843820e-06,2.574245e-06,0.000003


In [ ]:
# NorCPM1 historical: nans in lat_bnds

# Updated: NorESM, NorCPM, CESM2, GISS


In [47]:
model

'GISS-E2-1-G'

In [105]:
# check dataset availability:
#col_url = "https://raw.githubusercontent.com/NCAR/intake-esm-datastore/master/catalogs/pangeo-cmip6.json"
col_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(col_url)
#cat = col.search(experiment_id = 'piClim-4xCO2', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'piClim-control', source_id = 'BCC-ESM1', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'piClim-control', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
#cat = col.search(experiment_id = 'abrupt-0p5xCO2', source_id = 'CESM2', variable_id=['tas', 'rlut', 'rsut', 'rsdt'], table_id='Amon')
cat = col.search(experiment_id = 'piControl', source_id = 'CNRM-CM6-1', variable_id=['areacella'])
cat.df

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
23516,CMIP,CNRM-CERFACS,CNRM-CM6-1,piControl,r1i1p1f2,fx,areacella,gr,gs://cmip6/CMIP/CNRM-CERFACS/CNRM-CM6-1/piCont...,NaN,20180814


In [106]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True}, cdf_kwargs={'chunks': {}})

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 1 group(s)
